# Lab: MNIST Deep Autoencoder

Welcome back! In this lab, you will extend the shallow autoencoder you built in the previous exercise. The model here will have a deeper network so it can handle more complex images.

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt

## Prepare the Dataset

You will prepare the MNIST dataset just like in the previous lab.

In [5]:
def map_image(image, label):
  '''Normalizes and flattens the image. Returns image as input and label.'''
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0
  image = tf.reshape(image, shape=(784,))

  return image, image

In [9]:
# Load the train and test sets from TFDS

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024

train_dataset = tfds.load('mnist', as_supervised=True, split="train")
train_dataset = train_dataset.map(map_image)
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat()

test_dataset = tfds.load('mnist', as_supervised=True, split="test")
test_dataset = test_dataset.map(map_image)
test_dataset = test_dataset.batch(BATCH_SIZE).repeat()

## Build the Model

As mentioned, you will have a deeper network for the autoencoder. Compare the layers here with that of the shallow network you built in the previous lab.

In [27]:
def deep_autoencoder():
    '''Builds the encoder and decoder using Dense layers.'''
    # START YOUR CODE HERE
    encoder = tf.keras.layers.Dense(units=128, activation='relu')(inputs)
    encoder = tf.keras.layers.Dense(units=64, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(units=32, activation='relu')(encoder)
    
    # Decoder layers
    decoder = tf.keras.layers.Dense(units=64, activation='relu')(encoder)
    decoder = tf.keras.layers.Dense(units=128, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(units=784, activation='sigmoid')(decoder)
    # END YOUR CODE HERE
    return encoder, decoder

# set the input tensor
inputs = tf.keras.layers.Input(shape=(784,))

# get the encoder and decoder output
deep_encoder_output, deep_autoencoder_output = deep_autoencoder()

# setup the encoder because you will visualize its output later
deep_encoder_model = tf.keras.Model(inputs=inputs, outputs=deep_encoder_output)

# setup the autoencoder
deep_autoencoder_model = tf.keras.Model(inputs=inputs, outputs=deep_autoencoder_output)


## Compile and Train the Model

In [30]:
train_steps = 60000 // BATCH_SIZE

deep_autoencoder_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')
deep_auto_history = deep_autoencoder_model.fit(train_dataset, steps_per_epoch=train_steps, epochs=50)

Epoch 1/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.2815
Epoch 2/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.1359
Epoch 3/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1192
Epoch 4/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1123
Epoch 5/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1076
Epoch 6/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1041
Epoch 7/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.1014
Epoch 8/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0989
Epoch 9/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0971
Epoch 10/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0956
Epoch 11/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0941
Epoch 12/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0932
Epoch 13/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0923
Epoch 14/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0914
Epoch 15/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step

## Display sample results

See the results using the model you just trained.

In [32]:
def display_one_row(disp_images, offset, shape=(28, 28)):
  '''Display sample outputs in one row.'''
  for idx, test_image in enumerate(disp_images):
    plt.subplot(3, 10, offset + idx + 1)
    plt.xticks([])
    plt.yticks([])
    test_image = np.reshape(test_image, shape)
    plt.imshow(test_image, cmap='gray')


def display_results(disp_input_images, disp_encoded, disp_predicted, enc_shape=(8,4)):
  '''Displays the input, encoded, and decoded output values.'''
  plt.figure(figsize=(15, 5))
  display_one_row(disp_input_images, 0, shape=(28,28,))
  display_one_row(disp_encoded, 10, shape=enc_shape)
  display_one_row(disp_predicted, 20, shape=(28,28,))

In [ ]:
# take 1 batch of the dataset
test_dataset = test_dataset.take(1)

# take the input images and put them in a list
output_samples = []
for input_image, image in tfds.as_numpy(test_dataset):
      output_samples = input_image

# pick 10 random numbers to be used as indices to the list above
idxs = np.random.choice(BATCH_SIZE, size=10)

# get the encoder output
encoded_predicted = deep_encoder_model.predict(test_dataset)

# get a prediction for the test batch
deep_predicted = deep_autoencoder_model.predict(test_dataset)

# display the 10 samples, encodings and decoded values!
display_results(output_samples[idxs], encoded_predicted[idxs], deep_predicted[idxs])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
